<a href="https://colab.research.google.com/github/lamalyon/ProjectMillion/blob/main/FUNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""tools.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1URn10aLbaubeAoXolqz2k6ikN9QRGvGC
"""



import pandas as pd
import requests
import cfscrape
from io import BytesIO
import json
import numpy as np
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import time
import datetime
import pytz
import yfinance as yf
from IPython.display import clear_output
import os
# import streamlit as st
# from streamlit_autorefresh import st_autorefresh

def viewAll(df,row=20):
    pd.set_option('display.max_rows', row, 'display.max_columns',None)
    display(df)
    pd.set_option('display.max_rows',20)

# --------------------------------------------------------------------------------------------------------------------
def timeIt(start):
    second = round(time.time() - start)
    t = str(datetime.timedelta(seconds = second))
    return t

# --------------------------------------------------------------------------------------------------------------------
def extraWaiting(startTime, wait):
    seconds = time.time() - startTime
    timePass = datetime.timedelta(seconds = seconds).seconds
    remainTime = wait - timePass
    if remainTime < 0:
        extraWait = 0
    elif remainTime < wait:
        extraWait = remainTime
    return extraWait

pd.options.mode.chained_assignment = None  # default='warn'
np.set_printoptions(suppress=True)

# --------------------------------------------------------------------------------------------------------------------
def getDate(date=None):
    if date==None:
        date0 = datetime.date.today()
    else:
        date0 = datetime.date(date[0],date[1],date[2])

    if date0.weekday() == 0:
        date1 = date0 - datetime.timedelta(3)

    elif date0.weekday() == 5:
        date0 = date0 - datetime.timedelta(1)
        date1 = date0 - datetime.timedelta(1)

    elif date0.weekday() == 6:
        date0 = date0 - datetime.timedelta(2)
        date1 = date0 - datetime.timedelta(1)
    else:
        date1 = date0 - datetime.timedelta(1)    
    
    date0 = date0.strftime('%Y%m%d')
    date1 = date1.strftime('%Y%m%d')
    return date0,date1

# --------------------------------------------------------------------------------------------------------------------
def dataLoadLoop(stocklist,period='1d',interval='1m',name='dataM'):
    testTime = time.time()
    data = yf.download(tickers = stocklist,
                    period=period,
                    interval=interval,
                    group_by='ticker',
                    auto_adjust=False,
                    prepost=True,
                    threads=True,
                    proxy=None)
    data.to_csv('stockDB/{name}.csv'.format(name=name))
    print('load data in ',str(timeIt(testTime)), 'last load at ', time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

# --------------------------------------------------------------------------------------------------------------------
def scrapeFinviz():
    scraper = cfscrape.create_scraper()
    #Caching the Login Page.
    scraper.get("https://finviz.com/login.ashx")
    #Sending a POST Request to Login to the Account.
    scraper.post("https://finviz.com/login_submit.ashx", data={'email': 'lam.nguyentran@live.com', 'password': 'Lamalyon69', 'remember' : 'true'})
    out = {}
    # scraping the Finviz website-----------------------------------------------
    for i in ['v=111','v=131']:
        file_url= "https://elite.finviz.com/export.ashx?" + i
        r = scraper.get(file_url, stream= True)
        out[i] = pd.read_csv(BytesIO(r.content)).drop('No.',axis=1)
        time.sleep(1)
    #processing data -----------------------------------------------------------
    df1 = out['v=111']
    df1 = df1[['Ticker','Company','Sector','Industry','Country']].set_index('Ticker')
    df2 = out['v=131']
    df2 = df2.set_index('Ticker')
    df2.columns = ['MarketCap_M','Outstanding_M','Float_M','InsiderOwnPc','InsiderTransPc', 'InstOwnPc','InstTransPc','FloatShortPc','ShortRatio','AvgVolume_M','Price','ChangePc','Volume']
    #format percentage and million unit
    for i in ['InsiderOwnPc','InsiderTransPc', 'InstOwnPc','InstTransPc','FloatShortPc','ChangePc']:
        df2[i] = round((df2[i].str.strip('%').astype('float'))/100,2)
    df2['AvgVolume_M'] = df2['AvgVolume_M']/1000

    df= df1.join(df2)
    return df

# --------------------------------------------------------------------------------------------------------------------
def formatFinviz(data):
    df = data.copy()
    df['Vol/Float'] = round(df['Volume']/(df['Float_M']*1000000),3)
    df['AvgVol/Float'] = round(df['AvgVolume_M']/df['Float_M'],3)
    df['Vol/AvgVol'] = round(df['Volume']/(df['AvgVolume_M']*1000000),3)
    df['Money_M'] = round((df['Volume'] * df['Price'])/1000000,3)
    df['Money/Mkc'] = round((df['Money_M'] / df['MarketCap_M']),3)
    df['MkcGroup'] = pd.cut(df['MarketCap_M'], bins = [0,100,1000,100000,1000000, 100000000], labels = ['0-100M','100M-1B','1B-100B','100B-1T','1T-100T'])
    return df

# --------------------------------------------------------------------------------------------------------------------
def splitDateTime(df):
    # split datetime data into date and time
    # still keep the datetime for sorting
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df['D'] = df['Datetime'].dt.date
    df['T'] = df['Datetime'].dt.time

# --------------------------------------------------------------------------------------------------------------------
def formatData(data):
    df = data.copy()
    df.reset_index(inplace=True)
    splitDateTime(df)
    df.sort_values('Datetime',inplace=True,ignore_index=True)
    return round(df[['Datetime','Open','High','Low','Close','Price','Volume','D','T']],2)

# --------------------------------------------------------------------------------------------------------------------
def prepareFinviz():
    today = scrapeFinviz()
    today = formatFinviz(today)
    return today

# --------------------------------------------------------------------------------------------------------------------
def calGradient(s,roll=5,center=False):
    grad = ((s+1)/(s.shift()+1)).rolling(roll, center=center).mean() - 1
    return grad

# --------------------------------------------------------------------------------------------------------------------
def archiveData(data):
    ETtz = pytz.timezone('US/Eastern')
    ETtime = datetime.datetime.now().astimezone(ETtz)
    outTime = datetime.time(ETtime.hour,ETtime.minute) 
    if (outTime> datetime.time(16,1)) or (outTime < datetime.time(9,28)):
        if  datetime.time(16,1) <outTime < datetime.time(23,59):
            date = ETtime.date()
        elif datetime.time(0,0) <outTime<datetime.time(9,28):
            date = ETtime.date() - datetime.timedelta(days=1)
        print('Archive today data:', date)
        data.to_csv(date.strftime('%Y_%m_%d')+'.csv')    
    else:
        print('Not archive data yet')
        
# --------------------------------------------------------------------------------------------------------------------
def localExtrema(s, n=5):
    dfroll = s.rolling(n).mean()
    localMax = dfroll.iloc[argrelextrema(dfroll.values, np.greater, order=n)]
    localMin = dfroll.iloc[argrelextrema(dfroll.values, np.less, order=n)]
    return {'min':localMin, 'max':localMax}
    
# --------------------------------------------------------------------------------------------------------------------
def normalize(s,max):
    return s/(s.max()/max)
# --------------------------------------------------------------------------------------------------------------------
def calBSV(data,yesterday, finvizdata, clock = [9,30]):
    df = data.copy()

    datetimeIndex = pd.to_datetime(df.index)
    timeIndex = datetimeIndex.time

    df.set_index(timeIndex,inplace=True) # set time index to calculate growth from 9:30 am

    for i in df.columns.get_level_values(0).unique():
        df[i,'Ticker'] = i
        df[i,'Date'] = datetimeIndex.date
        df[i,'Time'] = datetimeIndex.time
        df[i,'MarketCap'] = finvizdata.loc[i,'MarketCap_M']*1000000
        df[i,'Float'] = finvizdata.loc[i,'Float_M']*1000000
        df[i,'AvgVol'] = finvizdata.loc[i,'AvgVolume_M']*1000000
        df[i,'InstOwnPc'] = finvizdata.loc[i,'InstOwnPc']
        df[i,'InstTransPc'] = finvizdata.loc[i,'InstTransPc']
        df[i,'FloatShortPc'] = finvizdata.loc[i,'FloatShortPc']
        df[i,'ShortRatio'] = finvizdata.loc[i,'ShortRatio']



        df[i,'VolSum'] = df[i,'Volume'].fillna(0).cumsum()
        df[i,'VolSum/AvgVol'] = round(df[i,'VolSum'] / (finvizdata.loc[i,'AvgVolume_M']*1000000),3)
        df[i,'VolSum/Float'] = round(df[i,'VolSum'] / (finvizdata.loc[i,'Float_M']*1000000),3)
        df[i,'VolSum/YesterdayVol'] = round(df[i,'VolSum'] / yesterday[i,'Volume'],3)
        df[i,'MoneyIn/Mkc'] = round(((df[i,'Close'].fillna(method='ffill') * df[i,'Volume'].fillna(0)) / (finvizdata.loc[i]['MarketCap_M']*1000000)).cumsum(),3)
        df[i,'Grad_VolSum/Float'] = calGradient(df[i,'VolSum/Float'])
        df[i,'Grad_MoneyIn/Mkc'] = calGradient(df[i,'MoneyIn/Mkc'])
        df[i,'Grad_VolSum/AvgVol'] = calGradient(df[i,'VolSum/AvgVol'])
        df[i,'Grad_VolSum/YesterdayVol'] = calGradient(df[i,'VolSum/YesterdayVol'])


        df[i,'Close'] = df[i,'Close'].fillna(method='ffill').fillna(method='bfill') # fill forward and backward empty close price
        df[i,'Open'] = df[i,'Open'].fillna(method='ffill').fillna(method='bfill') # fill forward and backward empty open price

        df[i,'Growth'] = ((df[i,'Close'])/(df[i,'Close'].loc[ datetime.time(clock[0],clock[1])]))-1 # calculate growth from 9:30am, index need to be in time

        df.loc[df[i,'Close'] >= df[i,'Open'], (i,'buy')] = df.loc[df[i,'Close'] >= df[i,'Open'], (i,'Volume')]
        df.loc[df[i,'Close'] < df[i,'Open'], (i,'sell')] = df.loc[df[i,'Close'] < df[i,'Open'], (i,'Volume')]


        df.loc[datetime.time(clock[0],clock[1]),(i,'buy')] = df.loc[datetime.time(clock[0],clock[1]),(i,'Volume')]
        df.loc[datetime.time(clock[0],clock[1]),(i,'sell')] = df.loc[datetime.time(clock[0],clock[1]),(i,'Volume')]

        df[i,'buyCum'] = df[i,'buy'].fillna(0).cumsum()
        df[i,'sellCum'] = df[i,'sell'].fillna(0).cumsum()

        df[i,'moneyBuy'] = (df[i,'buy'].fillna(0)*df[i,'Close']).cumsum()
        df[i,'moneySell'] = (df[i,'sell'].fillna(0)*df[i,'Close']).cumsum()

        df[i,'buyCum/float'] =round((df[i,'buyCum'])/(finvizdata.loc[i,'Float_M']*1000000),3)
        df[i,'sellCum/float'] =round((df[i,'sellCum'])/(finvizdata.loc[i,'Float_M']*1000000),3)

        df[i,'moneyBuy/mkc'] =round((df[i,'moneyBuy'])/(finvizdata.loc[i,'MarketCap_M']*1000000),3)
        df[i,'moneySell/mkc'] =round((df[i,'moneySell'])/(finvizdata.loc[i,'MarketCap_M']*1000000),3)

        df[i,'buyVSsell/Mkc'] = df[i,'moneyBuy/mkc'] - df[i,'moneySell/mkc']
        df[i,'buyVSsell/Float'] = df[i,'buyCum/float'] - df[i,'sellCum/float']

    df.set_index(datetimeIndex,inplace=True) # set index back to the original datetime

    return df

# --------------------------------------------------------------------------------------------------------------------
def calBuySellVol(tickers,finvizData,clock=[9,30]):
    
    df0 = finvizData.copy()
    todayDF = yf.download(tickers = tickers,
                        period='1d',
                        interval='1m',
                        start = None,
                        end = None,
                        group_by='ticker',
                        auto_adjust=False,
                        prepost=True,
                        threads=True,
                        proxy=None)
    yesterdayDF = yf.download(tickers = tickers,
                        period='2d',
                        interval='1d',
                        start = None,
                        end = None,
                        group_by='ticker',
                        auto_adjust=False,
                        prepost=True,
                        threads=True,
                        proxy=None)
    # yesterdayDF.set_index(pd.to_datetime(yesterdayDF.index).date, inplace = True)

    if len(tickers)==1:
        ticker = tickers[0]
        col = todayDF.columns.tolist()
        ticker = [ticker] * len(col)
        newIndex = tuple(zip(ticker,col))
        todayDF.columns = newIndex
        yesterdayDF.columns = newIndex
  
    # yesterday = datetime.date.today() - datetime.timedelta(days=1)
    # yesterdayDF = yesterdayDF.loc[yesterday]
    yesterdayDF = yesterdayDF.iloc[0]

    df = calBSV(todayDF,yesterdayDF, finvizdata=finvizData,clock=clock)

    return df

# --------------------------------------------------------------------------------------------------------------------
def nextNday(start, next=1):
    end = (datetime.datetime.strptime(start, '%Y-%m-%d') + datetime.timedelta(days=next)).date().strftime('%Y-%m-%d')
    return end

# --------------------------------------------------------------------------------------------------------------------
def googleConnect():
    from google.colab import auth
    auth.authenticate_user()
    import gspread
    from oauth2client.client import GoogleCredentials
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    return gc

# --------------------------------------------------------------------------------------------------------------------
def googleSheetConfig(url):    
    try:
        gsheets = gc.open_by_url(url)
        sheets = gsheets.worksheet('config').get_all_values()
    except:
        gc=googleConnect()
        gsheets = gc.open_by_url(url)
        sheets = gsheets.worksheet('config').get_all_values()
    
    config = pd.DataFrame(sheets[1:], columns=sheets[0])
    config.replace('', np.NaN, inplace=True)
    configTicker = config[config.columns[:3]]
    config = config[config.columns[3:]].astype(float).dropna(how='all')
    return configTicker, config

# --------------------------------------------------------------------------------------------------------------------
def plotLowLevel(data,shareY = True, graph='composite'):
    ticker = sorted(data.columns.get_level_values(0).unique().tolist())
    colnum = len(ticker)
    datetimeIndex = data.index
    timeIndex = data.index.time

    df = data.loc[timeIndex >= datetime.time(9,20)]

    fig = make_subplots(rows=5, cols=colnum,horizontal_spacing=0.025, vertical_spacing=0.05, subplot_titles=ticker, shared_yaxes=shareY,shared_xaxes=True)

    
    for i in ticker:
        col = ticker.index(i) + 1
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'sellCum/float'], name='sell',marker_color='crimson', fill='tozeroy',line=dict(width=0.5)),row=1,col=col)
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'buyCum/float'], name= 'buy',marker_color= 'dodgerblue', fill='tozeroy',line=dict(width=0.5)),row=1,col=col)
     
        # if graph=='composite':
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'buyVSsell/Mkc'], name=i+'_buyVSsell',marker_color= 'crimson', fill='tozeroy',line=dict(width=1)),row=4,col=col)
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'Growth'], name=i+'_Growth',marker_color= 'dodgerblue',fill='tozeroy',line=dict(width=0.5)),row=4,col=col)
    
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'VolSum/Float'], name=i+'_VolSum/Float',marker_color= 'crimson', fill='tozeroy',line=dict(width=1),hoverlabel= {'namelength' :-1}),row=5,col=col)
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'MoneyIn/Mkc'], name=i+'_MoneyIn/Mkc',marker_color= 'dodgerblue',fill='tozeroy',line=dict(width=0.5),hoverlabel= {'namelength' :-1}),row=5,col=col)
        

        
        # elif graph =='independent':
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'buyVSsell/Float'], name='BvsS/F',marker_color= 'crimson', fill='tozeroy',line=dict(width=0.5)),row=2,col=col)
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'buyVSsell/Mkc'], name='$in/Mkc',marker_color= 'dodgerblue', fill='tozeroy',line=dict(width=0.5)),row=2,col=col)
        
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'Growth'], name=i+'_Growth',marker_color= 'green', fill='tozeroy',line=dict(width=0.5)),row=3,col=col)        
        

        # fig.add_trace(go.Scatter(x=df.index,y=df[i,'VolGradiant'],name=i+'_Gradiant',marker_color= 'forestgreen',fill='tozeroy',line=dict(width=0.5)),row=4,col=col)

    fig.update_layout(height = 900,showlegend=False)
    fig.update_yaxes(title_text='buy & sell', row=1, col=1)
    fig.update_yaxes(title_text='BvS/F+BvS/Mkc', row=2, col=1)
    fig.update_yaxes(title_text='Growth', row=3, col=1)
    fig.update_yaxes(title_text='buy/sell & growth', row=4, col=1)
    fig.update_yaxes(title_text='volSum/Float & moneyIn/Mkc', row=5, col=1)


    return fig

# --------------------------------------------------------------------------------------------------------------------
def plotHighLevel(data,dataYear,config,tickers, log=False):
    figHL = make_subplots(rows=2, cols=5,horizontal_spacing=0.025, vertical_spacing=0.1,
                        subplot_titles=('VolSum/Float',
                                        'MoneyIn/Mkc',
                                        'OneYear Vol/Float',
                                        'buyVSsell/Mkc',
                                        'buyVSsell/Float',
                                        

                                        'VolSum/AvgVol',
                                        'VolSum/YesterdayVol',
                                        'OneYear Growth',
                                        'VolGradient LastMinutes',
                                        'Today Growth'                                        
                                        ))

    datetimeIndex = data.index
    timeIndex = data.index.time
    df = data.loc[timeIndex >= datetime.time(9,20)] # slice data from 9:29am onward
    lastMinutes =  datetimeIndex[-1] - pd.Timedelta(minutes=config['volGradLastMin'])

    # mark = dataYear.columns.get_level_values(0).unique()
    mark = tickers
    colorList = ['maroon','red','purple','green','navy','teal','chocolate','darkblue','darkcyan','darkgoldenrod'] # css color name
    listColor = colorList.copy()
    for i in sorted(df.columns.get_level_values(0).unique().tolist()):
        if len(listColor)==0:
                listColor = colorList.copy()
        if i in mark:
            color = listColor.pop(0)
        figHL.add_trace(go.Scatter(x=dataYear.index,y=dataYear[i,'Vol/Float'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=1,col=3)
        figHL.add_trace(go.Scatter(x=dataYear.index,y=dataYear[i,'Growth'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=2,col=3)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'Growth'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=2,col=5)

        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'VolSum/Float'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=True, line=dict(width=0.5)),row=1,col=1)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'MoneyIn/Mkc'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=1,col=2)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'VolSum/AvgVol'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=2,col=1)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'VolSum/YesterdayVol'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False, line=dict(width=0.5)),row=2,col=2)

        # figHL.add_trace(go.Scatter(x=df.index,y=df[i,'VolGradiant'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False,line=dict(width=0.5)),row=1,col=3)
        figHL.add_trace(go.Scatter(x=df.loc[df.index > lastMinutes].index,y=df.loc[df.index > lastMinutes][i,'Grad_VolSum/Float'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False,line=dict(width=0.5)),row=2,col=4)
        
        # figHL.add_trace(go.Scatter(x=df[i,'Float'].drop_duplicates(), y=df[i,'MarketCap'].drop_duplicates(), name=i,legendgroup=i, marker_color= color if i in mark else 'steelblue',showlegend=False, text=[i],mode="markers+text",textposition="bottom center" ),row=1,col=3)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'buyVSsell/Float'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False,line=dict(width=0.5)),row=1,col=5)
        figHL.add_trace(go.Scatter(x=df.index,y=df[i,'buyVSsell/Mkc'],name=i,legendgroup=i,marker_color= color if i in mark else 'steelblue',showlegend=False,line=dict(width=0.5)),row=1,col=4)

    figHL.update_layout(width=1500,height = 900)

    if log:
        figHL.update_yaxes(type="log", row=1, col=1)
        figHL.update_yaxes(type="log", row=1, col=2) 
        figHL.update_yaxes(type="log", row=2, col=1)
        figHL.update_yaxes(type="log", row=2, col=2)

        figHL.update_yaxes(type="log", row=1, col=4)
        figHL.update_yaxes(type="log", row=1, col=5)

    return figHL

# --------------------------------------------------------------------------------------------------------------------
def filterTickers(scan,config):
    volCur_vs_AvgVol = scan['Vol/AvgVol'] > config['Vol/AvgVol']
    money_vs_mkc = scan['Money/Mkc'] >= config['Money/Mkc']
    vol_vs_float = scan['Vol/Float'] > config['Vol/Float']
    country = scan['Country'] == 'USA'
    change = scan['ChangePc'] > config['ChangePc']

    HLshortList = scan.loc[(volCur_vs_AvgVol 
                            & money_vs_mkc
                            & change)
                            | vol_vs_float
                        # & country
                        ].index.tolist()
    return HLshortList

# --------------------------------------------------------------------------------------------------------------------
def dataOneYear(tickers: list, finvizdata):
    dataYear = yf.download(tickers = tickers,
                period='1y',
                interval='1h',
                group_by='ticker',
                auto_adjust=False,
                prepost=True,
                threads=True,
                proxy=None)
    
    dataYear = dataYear.fillna(method='ffill').fillna(method='bfill') # fill na with ffill and bfill

    if len(tickers)==1:
        ticker = tickers[0]
        col = dataYear.columns.tolist()
        ticker = [ticker] * len(col)
        newIndex = tuple(zip(ticker,col))
        dataYear.columns = newIndex        

    for i in dataYear.columns.get_level_values(0).unique():
        dataYear[i,'Growth'] = ((dataYear[i,'Close'])/(dataYear[i,'Close'].iloc[0])) -1 # calculate growth since one year
        dataYear[i,'MarketCap'] = finvizdata.loc[i,'MarketCap_M']*1000000
        dataYear[i,'Float'] = finvizdata.loc[i,'Float_M']*1000000
        dataYear[i,'AvgVol'] = finvizdata.loc[i,'AvgVolume_M']*1000000

        dataYear[i,'Vol/Float'] = dataYear[i,'Volume'] / dataYear[i,'Float']
    return dataYear

# --------------------------------------------------------------------------------------------------------------------
def plotHighLevel3D(data,y1,z1):
    timeIndex = data.index.time
    df = data.loc[timeIndex >= datetime.time(9,29)] # slice data from 9:30am onward
    figHL3d = make_subplots(rows=1, cols=1,specs=[[{'is_3d': True}]],horizontal_spacing=0.025, vertical_spacing=0.1)
    for i in df.columns.get_level_values(0).unique():
        figHL3d.add_trace(go.Scatter3d(x=df.index, y=df[i,y1], z=df[i,z1],name = i,legendgroup=i, showlegend=False, mode = 'lines',line=dict(width=2, color='steelblue')),row=1,col=1)
        # figHL3d.add_trace(go.Scatter3d(x=df.index, y=df[i,y2], z=df[i,z2],name = i,legendgroup=i, showlegend=True, mode = 'lines',line=dict(width=2, color='steelblue')),row=1,col=2)

    figHL3d.update_layout(width=800,height=700,autosize=False,scene=dict(camera=dict(up=dict(x=0,y=0,z=1),eye=dict(x=0,y=1,z=1,)),aspectratio = dict( x=1, y=1, z=1 ),aspectmode = 'manual'))
    return figHL3d


# --------------------------------------------------------------------------------------------------------------------
def plotHighLevelDot(data,tickers,tradeAmount):
    figHL = make_subplots(rows=2, cols=2,horizontal_spacing=0.05, vertical_spacing=0.1,
                    subplot_titles=('Float & Mkcap',
                                    'TradeAmount -> Stock & FloatRatio',

                                    'InstOwn & InstTrans',
                                    'FloatShort & ShortRatio',
                                    ))
    df = data.copy()
    df = df.fillna(method='ffill').fillna(method='bfill')
    mark = tickers
    colorList = ['maroon','red','purple','green','navy','teal','chocolate','darkblue','darkcyan','darkgoldenrod'] # css color name
    listColor = colorList.copy()
    for i in sorted(df.columns.get_level_values(0).unique().tolist()):
        if len(listColor)==0:
            listColor = colorList.copy()
        if i in mark:
            color = listColor.pop(0)
        Float = df[i,'Float'].drop_duplicates()
        Mkcap = df[i,'MarketCap'].drop_duplicates()
        tradeAmountInStock = round(tradeAmount / df[i,'Close'].iloc[-1])
        tradeAmountInFloatRatio = tradeAmountInStock / df[i,'Float'].iloc[-1]
        instOwn = df[i,'InstOwnPc'].drop_duplicates()
        instTrans = df[i,'InstTransPc'].drop_duplicates()
        floatShort = df[i,'FloatShortPc'].drop_duplicates()
        shortRatio = df[i,'ShortRatio'].drop_duplicates()
        figHL.add_trace(go.Scatter(x= Float, y= Mkcap, name=i,legendgroup=i, marker_color= color if i in mark else 'steelblue',showlegend=True, text=[i],mode="markers+text",textposition="bottom center" ),row=1,col=1)
        figHL.add_trace(go.Scatter(x=[tradeAmountInStock], y=[tradeAmountInFloatRatio], name=i,legendgroup=i, marker_color= color if i in mark else 'steelblue',showlegend=False, text=[i],mode="markers+text",textposition="bottom center" ),row=1,col=2)
        figHL.add_trace(go.Scatter(x=instOwn, y=instTrans, name=i,legendgroup=i, marker_color= color if i in mark else 'steelblue',showlegend=False, text=[i],mode="markers+text",textposition="bottom center" ),row=2,col=1)
        figHL.add_trace(go.Scatter(x=floatShort, y=shortRatio, name=i,legendgroup=i, marker_color= color if i in mark else 'steelblue',showlegend=False, text=[i],mode="markers+text",textposition="bottom center" ),row=2,col=2)
    figHL.update_layout(width=900,height = 900)
    figHL.update_yaxes(type="log", row=1, col=1)
    figHL.update_xaxes(type="log", row=1, col=1)
    figHL.update_yaxes(type="log", row=1, col=2)
    figHL.update_xaxes(type="log", row=1, col=2)
    figHL.update_yaxes(type="log", row=2, col=1)
    figHL.update_xaxes(type="log", row=2, col=1)
    figHL.update_yaxes(type="log", row=2, col=2)
    figHL.update_xaxes(type="log", row=2, col=2)

    return figHL

def plotLowLevelGradient(data,shareY = True, roll=5, base=100):
    ticker = sorted(data.columns.get_level_values(0).unique().tolist())
    colnum = len(ticker)
    datetimeIndex = data.index
    timeIndex = data.index.time

    df = data.loc[timeIndex >= datetime.time(9,29)]

    fig = make_subplots(rows=5, cols=colnum,horizontal_spacing=0.025, vertical_spacing=0.05, subplot_titles=ticker, shared_yaxes=shareY,shared_xaxes=True)

    
    for i in ticker:
        col = ticker.index(i) + 1
        fig.add_trace(go.Scatter(x=df.index, y = df[i,'Growth'], name= i,marker_color= 'dodgerblue', fill='tozeroy',line=dict(width=0.5)),row=1,col=col)
        row=1
        for feature in ['VolSum/Float', 'MoneyIn/Mkc', 'VolSum/AvgVol', 'VolSum/YesterdayVol']:
            grad = calGradient(df[i,feature], roll=roll, center=False)
            norm = normalize(df[i,feature], grad.max())
            row += 1
            fig.add_trace(go.Scatter(x=df.index, y = grad, name= i,marker_color= 'dodgerblue', fill='tozeroy',line=dict(width=0.5)),row=row,col=col)
            fig.add_trace(go.Scatter(x=df.index, y = norm, name= i,marker_color= 'crimson',line=dict(width=1)),row=row,col=col)

    fig.update_layout(width=1500,height = 900,showlegend=False)
    fig.update_yaxes(title_text='Growth', row=1, col=1)
    fig.update_yaxes(title_text='VolSum/Float', row=2, col=1)
    fig.update_yaxes(title_text='MoneyIn/Mkc', row=3, col=1)
    fig.update_yaxes(title_text='VolSum/AvgVol', row=4, col=1)
    fig.update_yaxes(title_text='VolSum/YesterdayVol', row=5, col=1)
    return fig

    
def tickerNotify(data,watch, value=0.002):
    grad = pd.DataFrame(data.loc(axis=1)[:,'Grad_VolSum/Float'].fillna(method='ffill').iloc[-1])
    grad.columns = ['val']
    grad.index = grad.index.get_level_values(0)
    notify = grad.loc[grad['val']>=value]
    notify = set(notify.index.tolist()) - set(watch)
    notify = list(notify)
    notify = ', '.join(notify)
    return notify

def chatBot(text):
    requests.get('https://api.telegram.org/bot5051022192:AAGTzQ32vdk4SsnamM_86i95Om3BUYHjpBM/sendMessage?chat_id=1140142963&text={message}'.format(message=text))

def authCode():
    y = int(datetime.date.today().strftime('%Y'))
    d = int(datetime.date.today().strftime('%d'))
    m = int(datetime.date.today().strftime('%m'))
    return (y*d*m)+(y+d+m)+(14*10*1985)+(14+10+1985)